# State Student Spending Case Study
#### Examining any relationship between student sucess or lack thereof with how much money the state has invested


### Necessary Steps for Data Story
#### 1) Clean the Data 
    a. Import Data
    b. Transform variables
#### 2) Exploratory Data Analysis
    a. Visualize the data
#### 3) Preprocess and Train Data
#### 4) Create Machine Learning Model
#### 5) Summarize Findings

In [1]:
#import necessary modules

import pandas as pd
import numpy as np

# 1a. Import Data

In [2]:
#import Graduation rate, Dropout rate, State Spending, Population, & State GDP data
Grad = pd.read_csv('Data/Completion1318edit.csv')
Drop = pd.read_csv('Data/Dropout1318edit.csv')
StateSpend = pd.read_excel('Data/StateSpendingStudents.xlsx')
Pop = pd.read_csv('Data/Pop10-19.csv')
GDP = pd.read_csv('Data/State Quarterly GDP 2005 - 2021 - Sheet1.csv')

In [3]:
#import tax data 
Tax13 = pd.read_csv('Data/2013 State Taxes.csv')
Tax14 = pd.read_csv('Data/2014 State Taxes.csv')
Tax15 = pd.read_csv('Data/2015 State Taxes.csv')
Tax16 = pd.read_csv('Data/2016 State Taxes.csv')
Tax17 = pd.read_csv('Data/2017 State Taxes.csv')
Tax18 = pd.read_csv('Data/2018 State Taxes.csv')
Tax19 = pd.read_csv('Data/2019 State Taxes.csv')


#Note to self to automate process:
#Try a dictionary. Start with empty dictionary. Wild cards to read itieratively * .csv

## Cleaning Tax Data

In [4]:
#Examing the column names: Need to change them to match the rest of the data
Tax13.columns

Index(['Unnamed: 0', 'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC',
       'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
       'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
       'VT', 'VA', 'WA', 'WV', 'WI', 'WY'],
      dtype='object')

In [5]:
#Dictionary to change state abbreviations to full state names
us_state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
    'District of Columbia': 'DC',
}

In [6]:
#Temporary reload of Tax Data
#import tax data 
Tax13 = pd.read_csv('Data/2013 State Taxes.csv')
Tax14 = pd.read_csv('Data/2014 State Taxes.csv')
Tax15 = pd.read_csv('Data/2015 State Taxes.csv')
Tax16 = pd.read_csv('Data/2016 State Taxes.csv')
Tax17 = pd.read_csv('Data/2017 State Taxes.csv')
Tax18 = pd.read_csv('Data/2018 State Taxes.csv')
Tax19 = pd.read_csv('Data/2019 State Taxes.csv')


#Note to self to automate process:
#Try a dictionary. Start with empty dictionary. Wild cards to read itieratively * .csv

# WHERE THE TROUBLE STARTS FOR TRYING TO CLEAN TAX*****

In [7]:
#Trying to make a function that will edit the files to the appropriate format
#Before I append them to a final file

def TaxEdit(df):
    df = df.rename(columns = us_state_abbrev)
    df = df.transpose()
    df = df.reset_index(drop = False)
    df.rename(columns = {'index': 'State'}, inplace = True)
    df = df.infer_objects() #I can't make it sum because I don't have dtype of numeric for my data
    return df

#Here is me attempting to force it to become numberic so I can sum
#The plan was to edit this. Try to make it a function or loop.
#I want to be able to sum the different tax categories save them as the appropriate year
#Then all frames will be cleaned, labeled with a year, and ready to append to final file.

Tax13Func = TaxEdit(Tax13)
Tax13E = Tax13Func.drop([0])
Tax13E.iloc[:,1:26] = Tax13E.iloc[:,1:26].astype('int64')
Tax13E['Total Tax'] =Tax13E.sum(axis = 1, skipna = True) 

Tax13E.head()

,State,0,1,2,3,4,5,6,7,8,...,16,17,18,19,20,21,22,23,24,Total Tax
1,Alabama,325652,2331676,174395,93,297958,530244,1557,737619,120110,...,14086,87908,4,3202520,382202,18,42155,119424,0,0.0
2,Alaska,99598,0,39194,8427,60236,41608,0,4295,69175,...,514,40522,8504,0,630941,0,0,4016966,0,0.0
3,Arizona,762651,6792635,68684,531,424369,781426,234,21013,315428,...,15747,126846,1973,3397707,662026,0,18902,29829,0,0.0
4,Arkansas,1022066,2837788,50656,36109,162962,455914,3113,0,237328,...,9917,121868,1649,2649577,402874,92,30190,80862,24623,0.0
5,California,1982208,33915885,357000,0,2242379,5492850,14088,676997,868703,...,416056,4202960,4027,66809000,7462000,0,0,37732,74978,0.0


## Below is how I was initially changing tax, but I decided to try the above to see if I could fix the data frames and add the year before appending the files.**** Not using code below

### Keeping the code below in case there is a way to append the files then add the yearly totals although I don't believe there is

In [8]:
TaxList = [Tax13, Tax14, Tax15, Tax16, Tax17, Tax18, Tax19]

#Append all the tax files into a single data frame.
TaxFinal = pd.DataFrame()

for df in TaxList:
    TaxFinal = TaxFinal.append(df)


#Rename State Abbreviations
TaxFinal = TaxFinal.rename(columns = us_state_abbrev)

## NEW NOTES
# First Step: Transpose the data as well as reset the index   

TaxFinal = TaxFinal.transpose()
TaxFinal = TaxFinal.reset_index(drop = False)
TaxFinal.rename(columns = {'index': 'State'}, inplace = True)

# Second step: Group by state
#TaxFinal = TaxFinal.groupby(['State'])

#Find Total Taxes
TaxFinal['Total Taxes'] = TaxFinal.sum(axis = 1, skipna = True)


#THIS IS INCORRECT

#Then use melt function on this single data frame.
#TaxFinal = TaxFinal.melt(id_vars = ['State'], var_name='Year', value_name = 'Taxes')

#Examine
print(TaxFinal.head())

        State            0            1       2      3       4       5     6  \
0  Unnamed: 0          T01          T09     T10    T11     T12     T13   T14   
1     Alabama       325652  2.33168e+06  174395     93  297958  530244  1557   
2      Alaska        99598            0   39194   8427   60236   41608     0   
3     Arizona       762651  6.79264e+06   68684    531  424369  781426   234   
4    Arkansas  1.02207e+06  2.83779e+06   50656  36109  162962  455914  3113   

        7       8  ...     16      17    18       19      20   21     22  \
0     T15     T16  ...    NaN     NaN   NaN      NaN     NaN  NaN    NaN   
1  737619  120110  ...  12078   72738     1  4194844  684296    X  47988   
2    4295   69175  ...  10719   68759  4303        X  332520    X      X   
3   21013  315428  ...     16  190015  1544  5357251  514264    X  18946   
4       0  237328  ...   9236  139083  2220  3012491  533915    0  46610   

       23      24 Total Taxes  
0     NaN     NaN         0.0 

## Cleaning Median Income Data

In [9]:
#Read in Median Income (Deleted out Standard Deviation as well as repetitive information from original file)
MedianInc = pd.read_csv('Data/Median Income Edit.csv')

#Flip
MedianInc = MedianInc.melt(id_vars = ['State'], var_name='Year', value_name = 'MedInc')

MedianInc.head()
MedianInc.tail()
#Delete unnecessary columns and repetitive information.

,State,Year,MedInc
1850,Virginia,1984,"26,525"
1851,Washington,1984,"25,017"
1852,West Virginia,1984,"16,843"
1853,Wisconsin,1984,"20,743"
1854,Wyoming,1984,"23,816"


## Cleaning Graduation and Drop Out Data

In [10]:
#examine data
print(Grad.columns)

Index(['State', '2012-2013', '2013-2014', '2014-2015', '2015-2016',
       '2016-2017', '2017-2018'],
      dtype='object')


In [11]:
#Rename columns so they match years in other files.
oldname = Grad.columns
newname = ['State', 2013, 2014, 2015, 2016, 2017, 2018 ]
Grad = Grad.rename(columns=dict(zip(oldname, newname)))

#Flip so States are columns
Grad = Grad.melt(id_vars = ['State'], var_name='Year', value_name = 'Graduates')


#Examine
Grad.head()

,State,Year,Graduates
0,United States,2013,81.4
1,Alabama,2013,80.0
2,Alaska,2013,71.8
3,Arizona,2013,75.1
4,Arkansas,2013,84.9


In [12]:
#examine data
print(Drop.head())
print(Drop.columns)

           State 2013 2014 2015  2016 2017 2018
0  United States  6.8  6.3  6.0   5.8  5.4  5.3
1        Alabama  8.9  7.7  7.4   6.9  6.6  5.4
2         Alaska  6.2  2.8  8.8   5.6  5.3  4.4
3        Arizona  9.4  8.5  9.2   7.6  8.0  7.7
4       Arkansas  6.8  6.8  6.7   7.2  5.7  5.6
Index(['State', '2013', '2014', '2015', '2016', '2017', '2018'], dtype='object')


In [13]:
#Flip so States are columns
Drop = Drop.melt(id_vars = ['State'], var_name='Year', value_name = 'Dropout')

#Examine
Drop.head()

,State,Year,Dropout
0,United States,2013,6.8
1,Alabama,2013,8.9
2,Alaska,2013,6.2
3,Arizona,2013,9.4
4,Arkansas,2013,6.8


## Clean State Spending Data

In [14]:
#Examine State Spending
print(StateSpend.head())

        state      1997      1998      1999      2000        2001      2002  \
0     Alabama   3271969   3504764   3784422   4006894   4140052.5   4273211   
1      Alaska   1042311   1019614   1097282   1133145   1188460.5   1243776   
2     Arizona   3388165   3579363   3849420   4185930   4427480.5   4669031   
3    Arkansas   1960613   2127517   2224967   2324123   2451202.0   2578281   
4  California  28708364  31776920  33739312  36658880  41025508.0  45392132   

         2003      2004      2005  ...      2007      2008      2009  \
0   4429788.5   4586366   4925201  ...   6104470   6671430   6629154   
1   1287792.0   1331808   1416771  ...   1609832   1897947   2077152   
2   5087547.5   5506064   5872686  ...   6826134   7367909   7536710   
3   2742677.5   2907074   3300288  ...   3719097   3846131   4189066   
4  47133896.0  48875660  50693044  ...  57258252  60096496  59255344   

       2010      2011      2012      2013      2014      2015      2016  
0   6573857   6592

In [15]:
#Realign with States as columns
StateSpend.rename(columns = {'state': 'State'}, inplace = True)
StateSpend = StateSpend.melt(id_vars = ['State'], var_name='Year', value_name = 'StateSpend')
StateSpend.head()

#Examine
print(StateSpend.head())

        State  Year  StateSpend
0     Alabama  1997   3271969.0
1      Alaska  1997   1042311.0
2     Arizona  1997   3388165.0
3    Arkansas  1997   1960613.0
4  California  1997  28708364.0


## Clean Population Data

In [16]:
#Examine the Population
print(Pop.head(10))
print(Pop['Geographic Area'].head(10))
#print(Pop.rows)

  Geographic Area       Census Estimates Base         2010         2011  \
0   United States  308,745,538    308,758,105  309,321,666  311,556,874   
1       Northeast   55,317,240     55,318,443   55,380,134   55,604,223   
2         Midwest   66,927,001     66,929,725   66,974,416   67,157,800   
3           South  114,555,744    114,563,030  114,866,680  116,006,522   
4            West   71,945,553     71,946,907   72,100,436   72,788,329   
5        .Alabama    4,779,736      4,780,125    4,785,437    4,799,069   
6         .Alaska      710,231        710,249      713,910      722,128   
7        .Arizona    6,392,017      6,392,288    6,407,172    6,472,643   
8       .Arkansas    2,915,918      2,916,031    2,921,964    2,940,667   
9     .California   37,253,956     37,254,519   37,319,502   37,638,369   

          2012         2013         2014         2015         2016  \
0  313,830,990  315,993,715  318,301,008  320,635,163  322,941,311   
1   55,775,216   55,901,806   56,0

In [17]:
#Fix geographic area names
Pop['Geographic Area'] = Pop['Geographic Area'].str.replace('.','')
print(Pop['Geographic Area'].head(10))

0    United States
1        Northeast
2          Midwest
3            South
4             West
5          Alabama
6           Alaska
7          Arizona
8         Arkansas
9       California
Name: Geographic Area, dtype: object


In [18]:
#Drop the US regions
Pop = Pop.drop([0,1,2,3,4,])
Pop = Pop.reset_index(drop = True)
Pop.tail(10)

,Geographic Area,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
41,South Dakota,"814,180","814,198","816,166","823,579","833,566","842,316","849,129","853,988","862,996","872,868","878,698","884,659"
42,Tennessee,"6,346,105","6,346,276","6,355,311","6,399,291","6,453,898","6,494,340","6,541,223","6,591,170","6,646,010","6,708,799","6,771,631","6,829,174"
43,Texas,"25,145,561","25,146,091","25,241,971","25,645,629","26,084,481","26,480,266","26,964,333","27,470,056","27,914,410","28,295,273","28,628,666","28,995,881"
44,Utah,"2,763,885","2,763,891","2,775,332","2,814,384","2,853,375","2,897,640","2,936,879","2,981,835","3,041,868","3,101,042","3,153,550","3,205,958"
45,Vermont,"625,741","625,737","625,879","627,049","626,090","626,210","625,214","625,216","623,657","624,344","624,358","623,989"
46,Virginia,"8,001,024","8,001,049","8,023,699","8,101,155","8,185,080","8,252,427","8,310,993","8,361,808","8,410,106","8,463,587","8,501,286","8,535,519"
47,Washington,"6,724,540","6,724,540","6,742,830","6,826,627","6,897,058","6,963,985","7,054,655","7,163,657","7,294,771","7,423,362","7,523,869","7,614,893"
48,West Virginia,"1,852,994","1,853,018","1,854,239","1,856,301","1,856,872","1,853,914","1,849,489","1,842,050","1,831,023","1,817,004","1,804,291","1,792,147"
49,Wisconsin,"5,686,986","5,687,285","5,690,475","5,705,288","5,719,960","5,736,754","5,751,525","5,760,940","5,772,628","5,790,186","5,807,406","5,822,434"
50,Wyoming,"563,626","563,775","564,487","567,299","576,305","582,122","582,531","585,613","584,215","578,931","577,601","578,759"


In [19]:
#Flip Columns
Pop.rename(columns = {'Geographic Area': 'State'}, inplace = True)
Pop = Pop.melt(id_vars = ['State'], var_name='Year', value_name = 'Population')
Pop.head()

,State,Year,Population
0,Alabama,Census,"4,779,736"
1,Alaska,Census,"710,231"
2,Arizona,Census,"6,392,017"
3,Arkansas,Census,"2,915,918"
4,California,Census,"37,253,956"


## Clean GDP  Data

## Not sure how to extract the year***

### Trying to move the columns frame down so that I can change the dtype and extract year
But am getting confused **

In [20]:
import pandas as pd
GDP = pd.read_csv('Data/State Quarterly GDP 2005 - 2021 - Sheet1.csv')

#Flip states to a column.
GDP = GDP.transpose()


#Trying to move states

GDP.columns = GDP.iloc[0,:]
GDP = GDP.iloc[1:, :]

#Need to rename columns
GDP = GDP.reset_index()

#Properly name State column
GDP = GDP.rename({'index': 'State'}, axis='columns')

GDP.head()

# Then use melt function
#GDPmelt = GDP.melt(id_vars = ['State'], var_name='QTR Year', value_name = 'State GDP')
#GDPmelt.head()

#Take average across 4 quarters for yearly GDP
### THIS CLEARLY WON'T WORK BECAUSE OF DTYPE ISSUES AND ETC, but this is where my mind is stuck

#for x in GDP.columns:
    #year = x.year
    #print(year)
    
#Use year extractor
#Add max/min/median as well to round it out

DATE,State,2005-01-01,2005-04-01,2005-07-01,2005-10-01,2006-01-01,2006-04-01,2006-07-01,2006-10-01,2007-01-01,...,2018-10-01,2019-01-01,2019-04-01,2019-07-01,2019-10-01,2020-01-01,2020-04-01,2020-07-01,2020-10-01,2021-01-01
0,Alabama,153078,155410,156810,160316,161733,163926,165313,166419,167130,...,224373,225565,226927,229328,230750,229831,209852,228062,231736,238726
1,Alaska,37393.9,38641.2,40259.9,42859.7,42664.7,44378.4,45039.4,45745.9,46468.8,...,54765.1,54137.6,54279.9,54449.9,54674.7,52864.4,45644.4,50412.9,52065,54628.8
2,Arizona,219880,225374,232290,235386,242418,244981,250031,255187,257837,...,357077,361380,366905,373173,379019,377476,350141,378296,383930,394490
3,Arkansas,87648.2,88484.4,89890.7,92549.3,92836,94824.6,95041.7,95104,94145.5,...,129333,129193,130471,131556,132596,131818,120812,130710,132956,137312
4,California,1.07133e+06,1.14794e+06,1.24773e+06,1.36617e+06,1.38755e+06,1.43934e+06,1.53056e+06,1.63243e+06,1.75265e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Clean Unemployment Rates

In [21]:
# Empty data frame
final_unemployment_data = pd.DataFrame()


for i in np.arange(2009,2018,1):
    print("Reading: {}".format(str(i)))
    
    year_data = pd.read_excel('Data/UnemploymentRateData_Parsed.xlsx', sheet_name = str(i))
    melt_data = pd.melt(year_data, id_vars = 'State', var_name = ['Month'], value_name = 'value')

    aggregated_data = melt_data.groupby('State').agg({'value' : ['min', 'max', 'mean']}).reset_index()
    aggregated_data.columns = ['State', 'min_unemploymentRate', 'max_unemploymentRate', 'mean_unemploymentRate']
    aggregated_data['Year'] = str(i)
    
# Logic to add each years data to the end of previously created data
final_unemployment_data = final_unemployment_data.append(aggregated_data)

final_unemployment_data.head()

Reading: 2009
Reading: 2010
Reading: 2011
Reading: 2012
Reading: 2013
Reading: 2014
Reading: 2015
Reading: 2016
Reading: 2017


,State,min_unemploymentRate,max_unemploymentRate,mean_unemploymentRate,Year
0,Alabama,3.8,5.5,4.408333,2017
1,Alaska,7.0,7.2,7.141667,2017
2,Arizona,4.7,5.2,4.858333,2017
3,Arkansas,3.6,3.7,3.666667,2017
4,California,4.5,5.2,4.775000,2017


## Merge ALL the clean files

In [ ]:
#Change them all to strings because??

Grad['Year'] = Grad['Year'].astype('str')
Grad['State'] = Grad['State'].astype('str')

Drop['Year'] = Drop['Year'].astype('str')
Drop['State'] = Drop['State'].astype('str')

StateSpend['Year'] = StateSpend['Year'].astype('str')
StateSpend['State'] = StateSpend['State'].astype('str')

Pop['Year'] = Pop['Year'].astype('str')
Pop['State'] = Pop['State'].astype('str')

In [ ]:
#merge graduate and drop data
grad_drop_merge = Grad.merge(Drop, on = ['Year', 'State'], how = 'left')
grad_drop_merge.head()

In [ ]:
#Merge state spending data
all_spend_merge = grad_drop_merge.merge(StateSpend, on = ['Year', 'State'], how = 'left')
all_spend_merge.head()

In [ ]:
# Merge in population
all_merge = all_spend_merge.merge(Pop, on = ['Year', 'State'], how = 'left')
all_merge.head()

# Begin Exploratory Data Analysis

In [ ]:
#Limit to years of interest
#StateSpendTrain = StateSpend.iloc[16:20]

#State Spend Documentation ends after 2016
#StateSpendTest = StateSpend.iloc[19:]

#print(StateSpendTrain)
#print(StateSpendTest)